In [2]:
import pandas as pd
import numpy as np
import re

# Direct link – works 100%
URL = "/content/fifa21_raw_data.csv"

print("Downloading FIFA 21 dataset...")
df = pd.read_csv(URL, low_memory=False)
print(f"Loaded: {df.shape[0]:,} players × {df.shape[1]} columns\n")

# ========================================
# FIXED & ROBUST CLEANING PIPELINE
# ========================================

# 1. Height → cm
def height_to_cm(h):
    if pd.isna(h):
        return np.nan
    h = str(h).strip().replace('"', '')
    if "'" in h:                              # e.g. 5'7"
        feet, inches = map(int, re.findall(r'\d+', h))
        return round(feet * 30.48 + inches * 2.54)
    try:
        return int(h) if h.isdigit() else np.nan   # rare cases already in cm
    except:
        return np.nan

df['Height'] = df['Height'].apply(height_to_cm)

# 2. Weight → kg
def weight_to_kg(w):
    if pd.isna(w):
        return np.nan
    w = str(w).strip()
    if 'lbs' in w:
        return int(re.search(r'\d+', w).group()) * 0.453592
    try:
        return float(w)
    except:
        return np.nan

df['Weight'] = df['Weight'].apply(weight_to_kg)

# 3. Money columns (€67.5M, €560K → float)
def money_to_numeric(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace('€', '').strip()
    multiplier = 1
    if val.endswith('M'):
        multiplier = 1_000_000
        val = val[:-1]
    elif val.endswith('K'):
        multiplier = 1_000
        val = val[:-1]
    try:
        return float(val) * multiplier
    except:
        return np.nan

for col in ['Value', 'Wage', 'Release Clause']:
    df[col] = df[col].apply(money_to_numeric)

# 4. Star ratings (W/F, SM, IR)
def clean_stars(s):
    if pd.isna(s):
        return np.nan
    # Extract only digits from the string
    digits = re.findall(r'\d+', str(s))
    if digits:
        return int(digits[0])
    return np.nan # If no digits are found

df['W/F'] = df['W/F'].apply(clean_stars)
df['SM']  = df['SM'].apply(clean_stars)
df['IR']  = df['IR'].apply(clean_stars)

# 5. Hits column (has quotes and sometimes "\n")
df['Hits'] = (df['Hits']
              .astype(str)
              .str.replace(r'[\",]', '', regex=True)
              .str.strip())
df['Hits'] = pd.to_numeric(df['Hits'], errors='coerce')

# 6. Joined date
df['Joined'] = pd.to_datetime(df['Joined'], errors='coerce')

# 7. Split Team & Contract properly (handles newlines correctly)
df['Team'] = df['Team & Contract'].str.split('\n').str[1].str.strip()
df['Contract'] = df['Team & Contract'].str.extract(r'(\d{4} ~ \d{4}|Free|On Loan.*?202\d|202\d ~ 202\d)', expand=False)

# 8. Drop messy original columns + URLs
df = df.drop(columns=['photoUrl', 'playerUrl', 'Team & Contract'], errors='ignore')

# 9. Force all known numeric columns to numeric
numeric_columns = [
    'Age', '↓OVA', 'POT', 'BOV', 'Growth',
    'Attacking', 'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
    'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
    'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
    'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
    'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
    'Defending', 'Marking', 'Standing Tackle', 'Sliding Tackle',
    'Goalkeeping', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
    'Total Stats', 'Base Stats', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY'
]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Clean column names
df.columns = df.columns.str.strip()

print("Cleaning completed successfully!")
print(f"Final shape: {df.shape[0]:,} × {df.shape[1]}")
print("\nFirst 5 rows:")
print(df.head())

# Optional: save
# df.to_csv("fifa21_cleaned.csv", index=False)

Loaded: 18,979 players × 77 columns

Cleaning completed successfully!
Final shape: 18,979 × 76

First 5 rows:
                       LongName Nationality Positions               Name  Age  \
0                  Lionel Messi   Argentina  RW ST CF           L. Messi   33   
1  C. Ronaldo dos Santos Aveiro    Portugal     ST LW  Cristiano Ronaldo   35   
2                     Jan Oblak    Slovenia        GK           J. Oblak   27   
3               Kevin De Bruyne     Belgium    CAM CM       K. De Bruyne   29   
4    Neymar da Silva Santos Jr.      Brazil    LW CAM          Neymar Jr   28   

   ↓OVA  POT      ID  Height     Weight  ... IR  PAC SHO  PAS DRI DEF  PHY  \
0    93   93  158023     170  72.121128  ...  5   85  92   91  95  38   65   
1    92   92   20801     188  83.007336  ...  5   89  93   81  89  35   77   
2    91   93  200389     188  87.089664  ...  3   87  92   78  90  52   90   
3    91   91  192985     180  69.853168  ...  4   76  86   93  88  64   78   
4    91   91 